### Data collection for various metrics:

Data Collection for Safety and Stability Metrics (Ignore the efficiency metrics even though they are displayed). 

Collection performed in 3 stages

    A. Data collection for Safety and CAV (Controller Acceleration Variation) 
    B. Data collevtion for WAR (Wave Attenuation Ratio)
    C. Data collection for Stability Plot

### Notes:


- Desired velocity change for FS May need monitoring (May not be set for all possible lengths)
- PI can have failures, so have to turn ON render and manually check
- LACC needs a different shock time (for no shock change time at eval)
- Generated data will be saved in the folder `test_time_rollout`
- We apply shocks after the systems have been stabilized
- Default shock times are between 8000 and 11000 unless explicitly specified otherwise
- Difference: For stability plot, the length is 270 whereas for WAR, the length is 260
- For WAR num_rollours is set to 1

### Penetration rates
- num_controlled = 1 (5%), 4(20%), 9(40%), 13(16%) 

### Changes that need to be made to files: 

    1. 
    2. For Singleagent, Efficiency at test time, make the accel behavior zero from apply_rl_actions in density_aware_env (single agent version)
    3. For Multiagent, Efficiency at train time. Make sure the leader is correct in multiagent_ring. Make sure all test time markers are turned off. Make sure to have correct number of NUM_AUTOMATED at 2 places
    4. For Multiagent, Efficiency at test time, make sure the correct exp_configs are present in the same folder as test_rllib.py. in controllers for daware, set efficiency to True for TrainedAgent. 
    5. For Multiagent, Safety + Stability at test time, followers zero acceleration behavior is set from apply_rl_actions in density_aware_env (multiagent version)


### A and B together: (Data collection for Safety and CAV) + (Data collection for WAR)

In [ ]:
# set parameters here
NUM_ROLLOUTS = 5
LENGTH = 260


### 1. (100%) IDM 

In [ ]:
!python classic.py --method idm --length $LENGTH --gen_emission --num_rollouts $NUM_ROLLOUTS --shock --render

In [ ]:
!python eval_metrics.py --method idm --save_plots

In [ ]:
# IDM does not need WAR, un-applicable

### 2. (5%) Single Vehicle Control

__FS__

In [ ]:
!python classic.py --method fs --length $LENGTH --gen_emission --num_rollouts $NUM_ROLLOUTS --shock --render --num_controlled 1

In [ ]:
!python eval_metrics.py --method fs --save_plots

In [ ]:
# Data for WAR
!python classic.py --method fs --stability --length $LENGTH --num_rollouts 1 --gen_emission --render --num_controlled 1

In [ ]:
# WAR
!python eval_plots.py --method fs 

__PIwS__
- This requires verifying if the particular run was `SUCCESS` or `FAIL`
- i.e., the controller may fail to stabilize the system

In [ ]:
# For CAV (first: get the data)
!python classic.py --method piws --length $LENGTH --gen_emission --num_rollouts $NUM_ROLLOUTS --shock --render --num_controlled 1

# Stable or not before shocks were applied
# 1. Stable 
# 2. Stable
# 3. Stable
# 4. Stable
# 5. Stable


In [ ]:
# use the eval_metrics code
!python eval_metrics.py --method pi --save_plots

In [ ]:
# Data for WAR (Check if it stabilized or not)
!python classic.py --method piws --stability --length $LENGTH --num_rollouts 1 --gen_emission --render --num_controlled 1

# Stable or not before shocks were applied
# 1. 
# 2. 
# 3. 
# 4. 
# 5. 


In [ ]:
!python eval_plots.py --method pi 

#### BCM

In [ ]:
# For CAV (first get the data)
!python classic.py --method bcm --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled 1

In [ ]:
# use the eval_metrics code
!python eval_metrics.py --method bcm --save_plots

In [ ]:
# No WAR. BCM is Unstable. But useful for later, to estimate in bottleneck.
!python classic.py --method bcm --stability --length $LENGTH --gen_emission\
--num_rollouts 5 --render --num_controlled 1

In [ ]:
!python eval_plots.py --method bcm 

#### LACC

In [ ]:
# For CAV (first get the data)
!python classic.py --method lacc --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --shock_start_time 11400 --shock_end_time 15000 --num_controlled 1


In [ ]:
# use the eval_metrics code
!python eval_metrics.py --method lacc --start_time 11400 --end_time 15000 --save_plots

In [ ]:
# No WAR. LACC is Unstable. But useful for later, to estimate in bottleneck.
# Data for WAR
!python classic.py --method lacc --stability --length $LENGTH --num_rollouts 5 --gen_emission\
--render --num_controlled 1 --shock_start_time 11400 --shock_end_time 15000

In [ ]:
# WAR
!python eval_plots.py --method lacc --start_time 11400 --end_time 15000 --propogate_time 200

__Wu et. al.__

In [ ]:
# For CAV (first get the data)
!python test_rllib.py ./Wu_et_al/Trained_policies/5_percent/PPO_WaveAttenuationPOEnv-v0_25b5cb6e_2022-01-26_10-58-12e9f4i3ao 50 \
--method wu --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled 1


In [ ]:
# For CAV
!python eval_metrics.py --method wu --save_plots

In [ ]:
# Data for WAR
!python test_rllib.py ./Wu_et_al/Trained_policies/5_percent/PPO_WaveAttenuationPOEnv-v0_25b5cb6e_2022-01-26_10-58-12e9f4i3ao 50\
--method wu --shock --stability --length $LENGTH --gen_emission --num_rollouts 1 --render --num_controlled 1


In [ ]:
!python eval_plots.py --method wu 

__Ours (1x)__
- The local zone is set to 50m at training. And it is set to 50m here as well.

In [ ]:
# For CAV (Safety + Stability)
!python test_rllib.py ./Ours/Trained_policies/5_percent/PPO_DensityAwareRLEnv-v0_5dfded14_2024-02-07_09-58-351869u4p3 168\
--method ours --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled 1


In [ ]:
# Only look at safety and stability
!python eval_metrics.py --method ours --save_plots

In [ ]:
# For CAV (Efficiency)

!python test_rllib.py ./Ours/Trained_policies/5_percent/PPO_DensityAwareRLEnv-v0_6106dcf6_2024-02-08_12-42-07_7vnf1n4 200\
--method ours --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled 1

In [ ]:
# Only look at Efficiency (Wipe out the previous data)
!python eval_metrics.py --method ours --save_plots

In [ ]:
# Data for WAR (Take the Safety + Stability Policy)
!python test_rllib.py ./Ours/Trained_policies/5_percent/PPO_DensityAwareRLEnv-v0_5dfded14_2024-02-07_09-58-351869u4p3 168\
--method ours --shock --stability --length $LENGTH --gen_emission --num_rollouts 1 --render --num_controlled 1


In [ ]:
!python eval_plots.py --method ours

### 3. (20%) Multiple Vehicle Control

### FS

In [ ]:
!python classic.py --method fs --length $LENGTH --gen_emission --num_rollouts $NUM_ROLLOUTS --shock --render --num_controlled 4


In [ ]:
# use the eval_metrics code
!python eval_metrics.py --method fs --save_plots

In [ ]:
# Data for WAR. Desired velocity should be set to 4.82 here
!python classic.py --method fs --stability --length $LENGTH --num_rollouts 5 --gen_emission\
--render --num_controlled 4


In [ ]:
# WAR
!python eval_plots.py --method fs 

__PIwS__

In [ ]:
!python classic.py --method piws --length $LENGTH --gen_emission --num_rollouts $NUM_ROLLOUTS --shock --render --num_controlled 4


In [ ]:
!python eval_metrics.py --method pi --save_plots

In [ ]:
# Data for WAR (Check if it stabilized or not)
!python classic.py --method piws --stability --length $LENGTH --num_rollouts 5 --gen_emission\
--render --num_controlled 4


In [ ]:
!python eval_plots.py --method pi 

__BCM__

In [ ]:
# For CAV (first get the data)
!python classic.py --method bcm --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled 4

In [ ]:
# use the eval_metrics code
!python eval_metrics.py --method bcm --save_plots

In [ ]:
# Data for WAR
!python classic.py --method bcm --stability --length $LENGTH --gen_emission\
--num_rollouts 5 --render --num_controlled 4


In [ ]:
!python eval_plots.py --method bcm 

__LACC__

In [ ]:
# For CAV (first get the data)
!python classic.py --method lacc --length $LENGTH --num_rollouts 5 --gen_emission\
--shock --render --num_controlled 4 --shock_start_time 11400 --shock_end_time 15000


In [ ]:
# use the eval_metrics code
!python eval_metrics.py --method lacc --start_time 11400 --end_time 15000 --save_plots

In [ ]:
# Unstable so no WAR

__Wu et. al. (20%)__

In [ ]:
# Agents are supposed to be dispersed.
# Stabalization time looks high, shocks can start later
!python test_rllib.py ./Wu_et_al/Trained_policies/20_percent/PPO_MultiAgentWaveAttenuationPOEnv-v1_436e49a2_2024-02-12_13-33-49wdl9scs5 174\
--method wu --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled 4


In [ ]:
# For CAV
!python eval_metrics.py --method wu --save_plots

In [ ]:
# Data for WAR (Take the Safety + Stability Policy)
!python test_rllib.py ./Wu_et_al/Trained_policies/20_percent/PPO_MultiAgentWaveAttenuationPOEnv-v1_436e49a2_2024-02-12_13-33-49wdl9scs5 174\
--method wu --shock --stability --length $LENGTH --gen_emission --num_rollouts 1 --render --num_controlled 4


In [ ]:
!python eval_plots.py --method wu

__Ours (20%)__

- The local zone of __leader__ is set to 50m at training. It is kept at 50m here as well.

In [ ]:
#### Safety + Stability ####
# For CAV (first get the data). Copy the exp_configs to a folder where test_rllib.py is located
# Make sure efficiency is set to False in controller
# Make sure the estimate is turned ON in multiagent env (in 2 places)
!python test_rllib.py ./Ours/Trained_policies/20_percent/PPO_MultiAgentDensityAwareRLEnv-v1_afca7452_2024-02-11_14-35-18pt1771a1 140\
--method ours4x --length $LENGTH --num_rollouts 20 --gen_emission --shock --render --num_controlled 4


In [ ]:
# Since !human is present in the code, this works seamlessly
!python eval_metrics.py --method ours4x --save_plots

In [ ]:
#### Efficiency ####
# Make sure to set the efficiency to true at test time in the controller file and have those files at exp_configs
# Make sure to have the right leader
!python test_rllib.py ./Ours/Trained_policies/20_percent/PPO_MultiAgentDensityAwareRLEnv-v1_33361ae0_2024-02-11_20-29-43f1pzfgrz 140\
--method ours4x --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled 4


In [ ]:
!python eval_metrics.py --method ours4x --save_plots

In [ ]:
# Data for WAR (taken on the Safety + Stability model). Make sure everything is tuned for Safety + Stability
!python test_rllib.py ./Ours/Trained_policies/20_percent/PPO_MultiAgentDensityAwareRLEnv-v1_afca7452_2024-02-11_14-35-18pt1771a1 140\
--method ours4x --shock --stability --length $LENGTH --gen_emission --num_rollouts 1 --render --num_controlled 4


In [ ]:
!python eval_plots.py --method ours4x

### 4. (40%) Multiple Vehicle Control

- 9 vehicles out of 22

In [ ]:
# set parameters here
NUM_ROLLOUTS = 5
LENGTH = 260
NUM_CONTROLLED = 9

### FS

In [ ]:
!python classic.py --method fs --length $LENGTH\
--gen_emission --num_rollouts $NUM_ROLLOUTS --shock --render --num_controlled $NUM_CONTROLLED

In [ ]:
# use the eval_metrics code
!python eval_metrics.py --method fs --save_plots

In [ ]:
# Data for WAR Desired velocity should be set to 4.82 here
!python classic.py --method fs --stability --length $LENGTH\
--num_rollouts 5 --gen_emission --render --num_controlled $NUM_CONTROLLED


In [ ]:
# WAR
!python eval_plots.py --method fs 

### PIwS

In [ ]:
!python classic.py --method piws --length $LENGTH\
--gen_emission --num_rollouts $NUM_ROLLOUTS --shock --render --num_controlled $NUM_CONTROLLED


In [ ]:
!python eval_metrics.py --method pi --save_plots

In [ ]:
# Data for WAR
!python classic.py --method piws --stability --length $LENGTH --num_rollouts 5 --gen_emission\
--render --num_controlled $NUM_CONTROLLED


In [ ]:
# WAR
!python eval_plots.py --method pi 

### BCM

In [ ]:
# For CAV (first get the data)
!python classic.py --method bcm --length $LENGTH\
--num_rollouts $NUM_ROLLOUTS --gen_emission --shock --num_controlled $NUM_CONTROLLED


In [ ]:
# use the eval_metrics code
!python eval_metrics.py --method bcm --save_plots


In [ ]:
# Data for WAR
!python classic.py --method bcm --stability --length $LENGTH --gen_emission\
--num_rollouts 5 --render --num_controlled $NUM_CONTROLLED


In [ ]:
# WAR
!python eval_plots.py --method bcm 

### LACC

In [ ]:
# For CAV (first get the data)
!python classic.py --method lacc --length $LENGTH\
--num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled $NUM_CONTROLLED --shock_start_time 11400 --shock_end_time 15000


In [ ]:
# use the eval_metrics code
!python eval_metrics.py --method lacc --start_time 11400 --end_time 15000 --save_plots


In [ ]:
# Data for WAR
!python classic.py --method lacc --stability --length $LENGTH --num_rollouts 5 --gen_emission\
--render --num_controlled $NUM_CONTROLLED --shock_start_time 11400 --shock_end_time 15000

In [ ]:
# WAR
!python eval_plots.py --method lacc --start_time 11400 --end_time 15000 --propogate_time 200

### Wu et. al. (40%)

In [ ]:
# Agents are supposed to be dispersed.
# Stabalization time looks high, shocks can start later
!python test_rllib.py ./Wu_et_al/Trained_policies/40_percent/PPO_MultiAgentWaveAttenuationPOEnv-v1_b8f3c414_2024-02-14_13-13-15_l2ifa89 92\
--method wu --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled 9


In [ ]:
# For CAV
!python eval_metrics.py --method wu --save_plots

In [ ]:
# Data for WAR

In [ ]:
# WAR

### Ours (40%)

In [ ]:
#### Safety + Stability ####
# For CAV (first get the data). Copy the exp_configs to a folder where test_rllib.py is located
# Make sure efficiency is set to False in controller
# Make sure the estimate is turned ON in multiagent env (in 2 places)
!python test_rllib.py ./Ours/Trained_policies/40_percent/PPO_MultiAgentDensityAwareRLEnv-v1_589d08ce_2024-02-14_13-03-242t0mb06j 60\
--method ours9x --length $LENGTH --num_rollouts 20 --gen_emission --shock --render --num_controlled $NUM_CONTROLLED



In [ ]:
# Since !human is present in the code, this works seamlessly
!python eval_metrics.py --method ours9x --save_plots

In [ ]:
#### Efficiency ####
# Make sure to set the efficiency to true at test time in the controller file and have those files at exp_configs
# Make sure to have the right leader
!python test_rllib.py ./Ours/Trained_policies/40_percent/PPO_MultiAgentDensityAwareRLEnv-v1_92538b0c_2024-02-15_14-36-53fn6l9uvs 140\
--method ours9x --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --render --shock --num_controlled $NUM_CONTROLLED


In [ ]:
# Since !human is present in the code, this works seamlessly
!python eval_metrics.py --method ours9x --save_plots

In [ ]:
# Data for WAR

In [ ]:
# WAR

### 5. (60%) Multiple Vehicle Control
- 13 vehicles controlled

In [ ]:
# set parameters here
NUM_ROLLOUTS = 5
LENGTH = 260
NUM_CONTROLLED = 13

### FS

In [ ]:
# Change get_desired_velocity velocity to 4.55 instead of 4.82
# Still it can get completely destabilitzed
!python classic.py --method fs --length $LENGTH\
--gen_emission --num_rollouts $NUM_ROLLOUTS --shock --render --num_controlled $NUM_CONTROLLED

In [ ]:
# use the eval_metrics code. May have to make 'fs' in tts calculation
!python eval_metrics.py --method fs --save_plots

In [ ]:
# Data for WAR. Change the desired velocity to 4.55
!python classic.py --method fs --stability --length $LENGTH\
--num_rollouts 5 --gen_emission --render --num_controlled $NUM_CONTROLLED


In [ ]:
# WAR
!python eval_plots.py --method fs 

### PIwS

In [ ]:
!python classic.py --method piws --length $LENGTH\
--gen_emission --num_rollouts $NUM_ROLLOUTS --shock --render --num_controlled $NUM_CONTROLLED


In [ ]:
!python eval_metrics.py --method pi --save_plots

In [ ]:
# Data for WAR
!python classic.py --method piws --stability --length $LENGTH --num_rollouts 5 --gen_emission\
--render --num_controlled $NUM_CONTROLLED


In [ ]:
# WAR
!python eval_plots.py --method pi 

### BCM

In [ ]:
# For CAV (first get the data)
!python classic.py --method bcm --length $LENGTH\
--num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled $NUM_CONTROLLED


In [ ]:
# use the eval_metrics code
!python eval_metrics.py --method bcm --save_plots


In [ ]:
# Data for WAR
!python classic.py --method bcm --stability --length $LENGTH --gen_emission\
--num_rollouts 5 --render --num_controlled $NUM_CONTROLLED


In [ ]:
# WAR
!python eval_plots.py --method bcm 

### LACC

In [ ]:
# For CAV (first get the data)
!python classic.py --method lacc --length $LENGTH\
--num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled $NUM_CONTROLLED --shock_start_time 11400 --shock_end_time 15000


In [ ]:
# use the eval_metrics code
!python eval_metrics.py --method lacc --start_time 11400 --end_time 15000 --save_plots


In [ ]:
# Data for WAR
!python classic.py --method lacc --stability --length $LENGTH --num_rollouts 5 --gen_emission\
--render --num_controlled $NUM_CONTROLLED --shock_start_time 11400 --shock_end_time 15000

In [ ]:
# WAR
!python eval_plots.py --method lacc --start_time 11400 --end_time 15000 --propogate_time 200

### Wu et. al. (60%)

In [ ]:
# Agents are supposed to be dispersed. Exp configs folder should be present outside where test_rllib.py is present
# Stabalization time looks high, shocks can start later
# 90 is highest but cannot stabilize at 90
!python test_rllib.py ./Wu_et_al/Trained_policies/60_percent/PPO_MultiAgentWaveAttenuationPOEnv-v1_24561808_2024-02-15_12-53-36bmm0ybn0 130\
--method wu --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled $NUM_CONTROLLED


# !python test_rllib.py ./Wu_et_al/Trained_policies/60_percent/PPO_MultiAgentWaveAttenuationPOEnv-v1_31346110_2024-02-15_12-53-57as6iehau 90\
# --method wu --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled $NUM_CONTROLLED



In [ ]:
!python eval_metrics.py --method wu --save_plots

In [ ]:
# Data for WAR

In [ ]:
# WAR

### Ours (60%)

In [ ]:
#### Safety + Stability ####
# For CAV (first get the data). Copy the exp_configs to a folder where test_rllib.py is located
# Make sure efficiency is set to False in controller
# Make sure the estimate is turned ON in multiagent env (in 2 places)
!python test_rllib.py ./Ours/Trained_policies/60_percent/PPO_MultiAgentDensityAwareRLEnv-v1_18eaa6f2_2024-02-16_15-29-348o30nqxq 200\
--method ours13x --length $LENGTH --num_rollouts $NUM_ROLLOUTS --gen_emission --shock --render --num_controlled $NUM_CONTROLLED


In [ ]:
!python eval_metrics.py --method ours13x --save_plots

In [ ]:
#### Efficiency ####
# Make sure to set the efficiency to true at test time in the controller file and have those files at exp_configs
# Make sure to have the right leader
!python test_rllib.py ./Ours/Trained_policies/60_percent/PPO_MultiAgentDensityAwareRLEnv-v1_ed546dfe_2024-02-14_17-18-06nijs5agx 200\
--method ours13x --length $LENGTH --num_rollouts 20 --render --gen_emission --shock --num_controlled $NUM_CONTROLLED


In [ ]:
!python eval_metrics.py --method ours13x --save_plots

In [ ]:
# Data for WAR

In [ ]:
# WAR

## C. Data collection for Stability Plot
- Execute the `eval_plots.py` code as well because this will create the stability plots (sometimes manual inspection is required)

In [ ]:
LENGTH_STABILITY = 270 #m
NUM_ROLLOUTS_STABILITY = 1

### IDM

In [ ]:
# This is for plot
# Stability shock is applied to IDM when the system is stable and SGT is not formed yet
# 150 is pretty early-on
!python classic.py --method idm --length $LENGTH_STABILITY --gen_emission --stability --shock_start_time 150 --shock_end_time 3710 --render

In [ ]:
# IDM is unstable (So there is no wave attenuation) - WAR does not apply
# But do we still need a value for reference (WAR = 0)
!python eval_plots.py --method idm --start_time 150 --end_time 3710 

### FS

In [ ]:
!python classic.py --method fs --stability --length $LENGTH_STABILITY --num_rollouts $NUM_ROLLOUTS_STABILITY --gen_emission --render

In [ ]:
!python eval_plots.py --method fs 

### PIwS

In [ ]:
# Data for WAR (Check if it stabilized or not)
!python classic.py --method piws --stability --length $LENGTH_STABILITY --num_rollouts $NUM_ROLLOUTS_STABILITY --gen_emission --render
# Check if was stable or not before the shock was applied

In [ ]:
!python eval_plots.py --method pi 

### RL+ LO (Wu et al.)

In [ ]:
!python test_rllib.py ./Wu_et_al/Trained_policies/PPO_WaveAttenuationPOEnv-v0_25b5cb6e_2022-01-26_10-58-12e9f4i3ao 50\
--method wu --shock --stability --length $LENGTH_STABILITY --gen_emission --num_rollouts $NUM_ROLLOUTS_STABILITY --render


In [ ]:
!python eval_plots.py --method wu 

### Ours (5%)
- Set local zone to 35m in the file before executing the code below

In [ ]:
# This data is taken with the local zone set to 35m

# Data for WAR (Take the Safety + Stability Policy)
!python test_rllib.py ./Ours/Trained_policies/5_percent/PPO_DensityAwareRLEnv-v0_5dfded14_2024-02-07_09-58-351869u4p3 168\
--method ours --shock --stability --length $LENGTH_STABILITY --gen_emission --num_rollouts 1 --render --num_controlled 1


In [ ]:
!python eval_plots.py --method ours

### BCM

In [ ]:
!python classic.py --method bcm --stability --length $LENGTH_STABILITY --num_rollouts $NUM_ROLLOUTS_STABILITY --gen_emission --num_rollouts $NUM_ROLLOUTS_WAR --render


In [ ]:
!python eval_plots.py --method bcm 

### Ours (20%)
- Set Local zone to 40m in the file before executing the code below 

In [ ]:
# This data is taken with the local zone set to 40m
# For CAV (first get the data). Copy the exp_configs to a folder where test_rllib.py is located
# Make sure efficiency is set to False in controller
# Make sure the estimate is turned ON in multiagent env (in 2 places)
# Data for WAR (taken on the Safety + Stability model). Make sure everything is tuned for Safety + Stability

!python test_rllib.py ./Ours/Trained_policies/20_percent/PPO_MultiAgentDensityAwareRLEnv-v1_afca7452_2024-02-11_14-35-18pt1771a1 140\
--method ours4x --shock --stability --length $LENGTH_STABILITY --gen_emission --num_rollouts 1 --render --num_controlled 4


In [ ]:
!python eval_plots.py --method ours4x

### LACC

In [ ]:
!python classic.py --method lacc --stability --length $LENGTH_STABILITY --num_rollouts $NUM_ROLLOUTS_STABILITY --gen_emission --shock_start_time 11400 --shock_end_time 15000 --render

In [ ]:
!python eval_plots.py --method lacc --start_time 11400 --end_time 15000 --propogate_time 200